<center><h1> IFT-6758  </h1></center>
<center><h1> Data Science / Science des données  </h1></center> 
<center><h2> Fall-2021 </h2></center> 
<center><h3> Prog 2 </h3></center> 
<center><h3> </h3></center> 



Deadline    : **Nov 20, 11.59 pm EDT** on [Gradescope](https://www.gradescope.com/courses/286503/assignments/1652286)

---

Date limite :  **Le 20 Nov, 23h59 HAE** sur [Gradescope](https://www.gradescope.com/courses/286503/assignments/1652286)

**Please use only the following imports.**

**DO NOT IMPORT ANYTHING OTHER THAN SUB-PACKAGES OF THESE WHEN NECESSARY.**

This is important for running you code!

---

**Veuillez utiliser uniquement les importations suivantes.**

**N'IMPORTEZ RIEN D'AUTRE DES SOUS-PACKAGES DE CEUX-CI LORSQUE NÉCESSAIRE.**

Ceci est important pour exécuter votre code!

In [180]:
# Imports / Importations

import numpy as np
import pandas as pd

from sklearn import preprocessing

from sklearn import feature_extraction 
from sklearn import feature_selection 

from sklearn import linear_model
from sklearn import cluster

from sklearn import model_selection
from sklearn import metrics

In [181]:
# Sample data inits / Initialisations des exemples de données

housing_raw = pd.read_csv('https://raw.githubusercontent.com/ift-6758/files/main/housing_raw.csv')

cluster_data = pd.read_csv('https://raw.githubusercontent.com/ift-6758/files/main/cluster-data.csv')

housing_processed = pd.read_csv('https://raw.githubusercontent.com/ift-6758/files/main/housing_processed.csv')

drug_trials = pd.read_csv('https://raw.githubusercontent.com/ift-6758/files/main/drug_trials.csv')

[4 points]

## Q1. Given a dataframe with housing data, perform the following preprocessing/feature engineering steps:

*   Remove the `Id` column.
*   Remove all non-numerical columns from the dataframe except `Neighborhood`.
*   
    If a column has >= 60% NaN values, remove the columns from the dataset.

    Otherwise, for numerical columns, impute those columns with the statistical median of the particular column.

*   Apply a Min Max Scaler to all the numerical features in the dataframe except `Neighborhood`.

*   Use the `DictVectorizer` to encode Neighborhood into one-hot feature representations and name these additional columns as `f0`,`f1`,`f2`..etc. to replace the `Neighborhood` column.

* Return the final dataframe.

---

## Q1. Étant donné un dataframe avec des données de logement, effectuez les étapes de prétraitement/d'ingénierie des caractéristiques suivantes :


* Supprimez la colonne `Id`.
* Supprimez toutes les colonnes non numériques de la base de données, à l'exception de `Neighborhood`.
*     Si une colonne a >= 60% valeurs NaN, supprimez les colonnes de l'ensemble de données.

     Sinon, pour les colonnes numériques, imputez à ces colonnes avec la médiane statistique de la colonne particulière.

* Appliquez un Min Max Scaler à toutes les caractéristiques numériques de la base de données, à l'exception de `Neighborhood`.

* Utilisez le `DictVectorizer` pour encoder Neighborhood en représentations one-hot de caractéristiques  et nommez ces colonnes supplémentaires comme `f0`,`f1`,`f2`..etc. pour remplacer la colonne `Neighborhood`.

* Retournez le dataframe finale.

In [183]:
def q1(df=housing_raw):
  """
  Your solution / Votre solution
  """
  df = df.drop('Id', 1)

  columns = df.columns

  for (i, dtype) in enumerate(df.dtypes):
    if not dtype == np.int64 and not dtype == np.float64 and not columns[i] == 'Neighborhood':
      df = df.drop(columns[i], 1)

  columns = df.columns
  for col in columns:
    na_ratio = df[col].isna().sum() / df.shape[0]
    if na_ratio >= 0.6:
      df = df.drop(col, 1)
    elif col != 'Neighborhood':
      df[col] = df[col].fillna(df[col].median())

  columns = df.columns
  for col in columns:
    if col != 'Neighborhood':
      if df[col].max() == df[col].min():
        df[col] = df[col] * 0.0
      else:
        df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())

  unique_count = df['Neighborhood'].unique().size
  col_arr = ['f' + str(i) for i in range(unique_count)]
  col = df['Neighborhood']
  neighborhood_dict_list = [{value: 1} for value in col]
  df[col_arr] = feature_extraction.DictVectorizer(sparse=False).fit_transform(neighborhood_dict_list)
  df = df.drop('Neighborhood', 1)
  return df

q1()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24
0,1.000000,0.164384,0.027026,0.444444,0.857143,0.303704,0.750000,0.000000,0.000000,0.0,0.318625,0.221719,0.296905,0.545657,0.0,0.484057,0.0,0.0,0.666667,0.0,0.833333,0.666667,0.8,0.000000,0.701923,0.00,0.000000,0.000000,0.000000,0.590674,0.0,0.0,0.0,0.0,0.545455,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.178082,0.049466,0.666667,0.428571,0.896296,0.766667,0.142235,0.362395,0.0,0.378077,0.486102,0.565764,0.000000,0.0,0.355299,0.5,0.0,0.666667,0.0,0.500000,0.333333,0.4,0.333333,0.865385,0.50,0.352507,0.134189,0.120658,0.000000,0.0,0.0,0.0,0.0,0.454545,0.75,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.099315,0.037884,0.444444,1.000000,0.533333,0.966667,0.000000,0.000000,0.0,0.324199,0.225598,0.176015,0.000000,0.0,0.110538,0.0,0.5,0.333333,0.0,0.333333,0.333333,0.2,0.000000,0.701923,0.00,0.000000,0.000000,0.365631,0.000000,0.0,0.0,0.0,0.0,0.454545,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.143836,0.030732,0.666667,0.428571,0.888889,0.766667,0.027576,0.000000,0.0,0.574547,0.399806,0.444391,0.000000,0.0,0.279077,0.0,0.0,0.666667,0.0,0.500000,0.333333,0.4,0.333333,0.855769,0.50,0.296460,0.256709,0.038391,0.000000,0.0,0.0,0.0,0.0,0.454545,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.058824,0.065068,0.016359,0.333333,0.714286,0.303704,0.666667,0.000000,0.103466,0.0,0.464004,0.386555,0.416828,0.000000,0.0,0.261767,0.5,0.0,0.333333,0.0,0.333333,0.333333,0.3,0.000000,0.490385,0.50,0.324484,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.545455,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0.294118,0.202055,0.071327,0.555556,0.857143,0.422222,0.000000,0.000000,0.264181,0.0,0.208546,0.307692,0.404255,0.654232,0.0,0.610689,0.0,0.0,0.666667,0.5,0.666667,0.333333,0.7,0.333333,0.250000,0.50,0.265487,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.545455,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
496,0.294118,0.130137,0.043890,0.444444,0.714286,0.333333,0.783333,0.000000,0.239496,0.0,0.100325,0.217195,0.163443,0.374165,0.0,0.306711,0.0,0.0,0.333333,0.0,0.500000,0.333333,0.4,0.000000,0.557692,0.25,0.345133,0.000000,0.234004,0.564767,0.0,0.0,0.0,0.0,0.545455,0.75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
497,0.000000,0.191781,0.032882,0.444444,0.428571,0.585185,0.066667,0.000000,0.000000,0.0,0.000000,0.000000,0.382012,0.000000,0.0,0.239903,0.0,0.0,0.333333,0.0,0.500000,0.333333,0.3,0.333333,0.701923,0.00,0.000000,0.000000,0.065814,0.492228,0.0,0.0,0.0,0.0,0.272727,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
498,0.000000,0.263699,0.047573,0.777778,0.428571,0.977778,0.966667,0.145138,0.639706,0.0,0.225732,0.550743,0.662476,0.000000,0.0,0.416034,0.5,0.0,0.666667,0.0,0.500000,0.333333,0.5,0.333333,0.980769,0.75,0.569322,0.000000,0.091408,0.000000,0.0,0.0,0.0,0.0,0.363636,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [185]:
col = housing_raw['Neighborhood']
neighborhood_dict_list = [{value: 1} for value in col]
#housing_raw['Neighborhood'].T.to_dict().values()
col.unique().size

25

[4 points]

## Q2. Given a dataframe with multiple features `f0`, `f1`, ..., apply KMeans for the given values of `k` and return the value of `k` with the **least** `inertia` of the clustering and this least intertia value.


---

## Q2. Étant donné un dataframe avec plusieurs caractéristiques `f0`, `f1`, ..., appliquez KMeans pour les valeurs données de `k` et retournez la valeur de `k` avec la **plus petite** `inertia` du clustering et cette plus petite valeur d'intertie.

In [186]:
def q2(df = cluster_data, k_values=[2,3,4,5]):
  """
  Your solution / Votre solution
  """
  best_k = None
  best_inertia = float('inf')

  for k in k_values:
    model = cluster.KMeans(n_clusters=k)
    model.fit(cluster_data)
    if model.inertia_ < best_inertia:
      best_inertia = model.inertia_
      best_k = k

  return best_k, best_inertia

q2()

(5, 4673.560999656434)

[4 points]

## Q3. Given a dataframe with housing data to predict the `SalePrice` value, return the best features (NOT `SalePrice`) using Lasso feature selection method. Use [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?highlight=lasso#sklearn.linear_model.Lasso) and [`SelectFromModel`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) to achieve this.

* Input  : `df` - dataframe and `alpha` parameter for `Lasso`
* Return : Set (`set()`) of columns with the feature names selected by Lasso

---

## Q3. Étant donné un dataframe des données de logement pour prédire la valeur `SalePrice` (prix de vente), retournez les meilleures caractéristiques (PAS `SalePrice`) en utilisant la méthode de sélection de caractéristiques Lasso. Utilisez  [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html?highlight=lasso#sklearn.linear_model.Lasso) et [`SelectFromModel`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html#sklearn.feature_selection.SelectFromModel) pour y parvenir.

* Entrée : `df` - dataframe et paramètre `alpha` pour `Lasso`
* Retour : Ensemble (`set()`) des colonnes avec les noms des caractéristiques sélectionnées par Lasso

In [187]:
def q3(df = housing_processed, alpha=0.1, target='SalePrice'):
  """
  Your solution / Votre solution
  """
  X = df.drop(target, 1)
  y = df[target]

  model = linear_model.Lasso(alpha=alpha)
  selector = feature_selection.SelectFromModel(estimator=model)
  selector.fit(X, y)
  selected = selector.get_feature_names_out(X.columns)
  return set(selected)

q3()

{'1stFlrSF',
 '2ndFlrSF',
 '3SsnPorch',
 'BedroomAbvGr',
 'BsmtFinSF1',
 'BsmtFinSF2',
 'BsmtFullBath',
 'BsmtHalfBath',
 'BsmtUnfSF',
 'EnclosedPorch',
 'Fireplaces',
 'FullBath',
 'GarageArea',
 'GarageCars',
 'GarageYrBlt',
 'GrLivArea',
 'HalfBath',
 'KitchenAbvGr',
 'LotArea',
 'LotFrontage',
 'LowQualFinSF',
 'MSSubClass',
 'MasVnrArea',
 'MiscVal',
 'MoSold',
 'OpenPorchSF',
 'OverallCond',
 'OverallQual',
 'PoolArea',
 'ScreenPorch',
 'TotRmsAbvGrd',
 'TotalBsmtSF',
 'WoodDeckSF',
 'YearBuilt',
 'YearRemodAdd',
 'YrSold'}

In [188]:
print(housing_processed.shape[1], len(q3()))

37 36


[4 points]

##Q4. Given a dataframe with housing data to predict the `SalePrice` value, return the average value of the validation mean squared error and `alpha` value of the **best** `Ridge` model fit in a $k$-fold Cross Validation setting based on a given set of `alpha` values (of the `Ridge` model). 
 
---

##Q4. Étant donné un dataframe avec des données de logement pour prédire la valeur `SalePrice`, retournez la valeur moyenne de l'erreur quadratique moyenne de validation et la valeur `alpha` du **meilleur** modèle `Ridge` ajusté dans une validation croisée $k$-fold basé sur une liste de valeurs `alpha` (du modèle `Ridge`). 

In [190]:
def q4(df = housing_processed, alphas=[0.1,0.01,0.001], k=5, target='SalePrice'):
  """
  Your solution / Votre solution
  """
  best_kfold_valid_mse = float('inf') 
  best_alpha = None
  X = df.drop(target, 1).to_numpy()
  y = df[target].to_numpy()

  for alpha in alphas:
    kf = model_selection.KFold(n_splits=k)
    mse_list = []
    for train_index, test_index in kf.split(X):
      X_train, y_train = X[train_index, :], y[train_index]
      X_test, y_test = X[test_index, :], y[test_index]
      model = linear_model.Ridge(alpha=alpha)
      model.fit(X_train, y_train)
      pred = model.predict(X_test)
      mse = ((pred - y_test) ** 2).mean()
      mse_list.append(mse)

    avg_mse = sum(mse_list) / len(mse_list)
    if(avg_mse < best_kfold_valid_mse):
      best_kfold_valid_mse = avg_mse
      best_alpha = alpha

  return best_kfold_valid_mse, best_alpha

q4()

(2115226330.6302967, 0.1)

[4 points] 

## Q5. Given a dataframe about results from a drug trial with two drugs (`drug_type` `0` and `1`) that aim to improve longevity (`life_expectancy`), use bootstrap estimation to find the confidence interval of difference in mean value of `life_expectancy` of each of these two drugs. 

* Use the pandas [`Dataframe.sample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) function to sample with replacement, with the bootstrap sample size as the length of the source dataframe and random state passed in the function argument.
* Use `drug_type 0` - `drug_type 1`order for the difference.

**Arguments**

* `df` : a dataframe that includes observations of the two sample classes
* `variable` : the column name of the column that includes observations of the variable of interest  
* `class_name` : the column name of the column that includes group assignment (This column should contain two different group names/values)
* `num_repetitions`: number of times you want the bootstrapping to repeat. Default is 1000.
* `alpha` : likelihood that the true population parameter lies outside the confidence interval. Default is 0.05. 
* `random_state` : enable users to set their own random_state for the sampling, default is `123`. 


**Return**

* The lower limit and upper limit of the confidence interval for the difference in mean `life_expectancy` between the two drug types corresponding to the given `alpha`.


---

## Q5. Étant donné un dataframe sur les résultats d'un essai de médicament avec deux médicaments (`drug_type` `0` et `1`) qui visent à améliorer la longévité (`life_expectancy`/espérance de vie), utilisez l'estimation bootstrap de la différence pour trouver l'intervalle de confiance de la valeur moyenne de la différence de `life_expectancy` de chacun de ces deux médicaments.

* Utilisez la fonction pandas [`Dataframe.sample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html) pour échantillonner avec remplacement, avec la taille de l'échantillon bootstrap comme longueur du dataframe source et l'état aléatoire transmis dans l'argument de la fonction.
* Utilisez l'ordre `drug_type 0` - `drug_type 1` pour la différence.

**Arguments**

* `df` : un dataframe qui comprend les observations des deux classes d'échantillons
* `variable` : le nom de la colonne qui inclut les observations de la variable d'intérêt
* `class_name` : le nom de colonne de la colonne qui inclut les balises de groupe (cette colonne doit contenir deux noms/valeurs de groupe différents)
* `num_repetitions`: le nombre de fois qu'on souhaite que le bootstrapping se répète. La valeur par défaut est 1000.
* `alpha` : probabilité que le véritable paramètre de population se situe en dehors de l'intervalle de confiance. La valeur par défaut est 0,05. 
* `random_state` : pour permettre aux utilisateurs de définir leur propre état aléatoire , la valeur par défaut est `123`


**Retour**

* La limite inférieure (lower limit) et la limite supérieure (upper limit) de l'intervalle de confiance pour la différence de `life_expectancy` (espérance de vie) moyenne entre les deux types de médicaments (drugs) correspondant à l'`alpha` donné.



In [191]:
def q5(df=drug_trials, variable='life_expectancy', class_name='drug_type', num_repetitions = 1000, alpha = 0.05, random_state=np.random.RandomState(seed=123)):
    """
    Your solution / Votre solution
    """
    uniq = np.sort(df[class_name].unique())
    diffs = []
    n = df.shape[0]

    for i in range(num_repetitions):
      sample = df.sample(n=n, replace=True, random_state=random_state)
      sample_0 = sample[sample[class_name] == uniq[0]][variable]
      sample_1 = sample[sample[class_name] == uniq[1]][variable]
      diff = sample_0.mean() - sample_1.mean()
      diffs.append(diff)

    lower_p = alpha / 2
    higher_p = 1 - alpha / 2
    lower_limit = np.quantile(diffs, lower_p)
    higher_limit = np.quantile(diffs, higher_p)

    return lower_limit, higher_limit

q5()

(-9.702295489130108, -2.49436924305019)

Packaging all the above functions into a class for the solution file to submit on Gradescope.

---

Empaqueter toutes les fonctions ci-dessus dans une classe pour le fichier de solution à remettre sur Gradescope.

In [192]:
class Prog2:
  
  def q1(self, df=housing_raw):
    return q1(df)
  
  def q2(self, df = cluster_data, k_values=[2,3,4,5]):
    return q2(cluster_data, k_values)
  
  def q3(self, df = housing_processed, alpha=0.1, target='SalePrice'):
    return q3(df, alpha, target)

  def q4(self, df = housing_processed, alphas=[0.1,0.01,0.001], k=5, target='SalePrice'):
    return q4(df, alphas, k, target)

  def q5(self, df=drug_trials, variable='life_expectancy', class_name='drug_type', num_repetitions = 1000, alpha = 0.05, random_state=np.random.RandomState(seed=123)):
    return q5(df, variable, class_name, num_repetitions, alpha, random_state)

In [193]:
p = Prog2()
print(p.q1())
print(p.q2())
print(p.q3())
print(p.q4())
print(p.q5())

     MSSubClass  LotFrontage   LotArea  OverallQual  ...  f21  f22  f23  f24
0      1.000000     0.164384  0.027026     0.444444  ...  0.0  0.0  0.0  0.0
1      0.000000     0.178082  0.049466     0.666667  ...  0.0  0.0  0.0  0.0
2      0.000000     0.099315  0.037884     0.444444  ...  0.0  0.0  0.0  0.0
3      0.000000     0.143836  0.030732     0.666667  ...  0.0  0.0  0.0  0.0
4      0.058824     0.065068  0.016359     0.333333  ...  0.0  0.0  0.0  0.0
..          ...          ...       ...          ...  ...  ...  ...  ...  ...
495    0.294118     0.202055  0.071327     0.555556  ...  0.0  0.0  0.0  0.0
496    0.294118     0.130137  0.043890     0.444444  ...  0.0  0.0  0.0  0.0
497    0.000000     0.191781  0.032882     0.444444  ...  0.0  0.0  0.0  0.0
498    0.000000     0.263699  0.047573     0.777778  ...  0.0  0.0  0.0  0.0
499    0.411765     0.047945  0.037860     0.555556  ...  0.0  0.0  0.0  0.0

[500 rows x 61 columns]
(5, 4673.703114197391)
{'MoSold', '1stFlrSF', 'Bsmt